# Cálculo de ratios

Cálculo de los ratios financieros a partir de la información contable y bursátil disponible, que utilizaremos como variables explicativas del modelo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 50

In [3]:
! ls -n ../tablas

total 12480
-rw-rw-r-- 1 1000 1000 9992731 ago 28 01:03 banks_cotiz.csv
-rw-rw-r-- 1 1000 1000  984636 ago 28 12:05 dataformodel.csv
-rw-rw-r-- 1 1000 1000  578247 ago 28 01:03 mergedbalres.csv
-rw-rw-r-- 1 1000 1000  683261 ago 28 01:03 registros_etiq.csv
-rw-rw-r-- 1 1000 1000  525808 ago 30 22:58 registros_etiq_normal.csv
-rw-rw-r-- 1 1000 1000    1997 ago 30 22:58 var_indice.csv


In [4]:
data = pd.read_csv('../tablas/registros_etiq_normal.csv', usecols=['Ticker',
                                                             'Fiscal Year',
                                                             'Fiscal Period',
                                                             'Publish Date',
                                                             'Shares (Basic)',
                                                             'Shares (Diluted)',
                                                             'Cash, Cash Equivalents & Short Term Investments',
                                                             'Interbank Assets',
                                                             'Short & Long Term Investments',
                                                             'Accounts & Notes Receivable',
                                                             'Net Loans',
                                                             'Net Fixed Assets',
                                                             'Total Assets',
                                                             'Total Deposits',
                                                             'Short Term Debt',
                                                             'Long Term Debt',
                                                             'Total Liabilities',
                                                             'Preferred Equity',
                                                             'Share Capital & Additional Paid-In Capital',
                                                             'Treasury Stock',
                                                             'Retained Earnings',
                                                             'Total Equity',
                                                             'Total Liabilities & Equity',
                                                             'Revenue',
                                                             'Provision for Loan Losses',
                                                             'Net Revenue after Provisions',
                                                             'Total Non-Interest Expense',
                                                             'Operating Income (Loss)',
                                                             'Non-Operating Income (Loss)',
                                                             'Pretax Income (Loss)',
                                                             'Income Tax (Expense) Benefit, Net',
                                                             'Income (Loss) from Continuing Operations',
                                                             'Net Extraordinary Gains (Losses)',
                                                             'Net Income',
                                                             'Net Income (Common)',
                                                             'Periodo',
                                                             'Price_d',
                                                             'Price_d_100',
                                                             'Publish_Date+180',
                                                             'Price_d+180_100',
                                                             'Price_d+180',
                                                             'Indice_d_100',
                                                             'Indice_d+180_100',
                                                             'Var_cotiz',
                                                             'Var_index',
                                                             'Etiqueta'])

In [5]:
len(data)

1021

In [6]:
list(data.columns)

['Ticker',
 'Fiscal Year',
 'Fiscal Period',
 'Publish Date',
 'Shares (Basic)',
 'Shares (Diluted)',
 'Cash, Cash Equivalents & Short Term Investments',
 'Interbank Assets',
 'Short & Long Term Investments',
 'Accounts & Notes Receivable',
 'Net Loans',
 'Net Fixed Assets',
 'Total Assets',
 'Total Deposits',
 'Short Term Debt',
 'Long Term Debt',
 'Total Liabilities',
 'Preferred Equity',
 'Share Capital & Additional Paid-In Capital',
 'Treasury Stock',
 'Retained Earnings',
 'Total Equity',
 'Total Liabilities & Equity',
 'Revenue',
 'Provision for Loan Losses',
 'Net Revenue after Provisions',
 'Total Non-Interest Expense',
 'Operating Income (Loss)',
 'Non-Operating Income (Loss)',
 'Pretax Income (Loss)',
 'Income Tax (Expense) Benefit, Net',
 'Income (Loss) from Continuing Operations',
 'Net Extraordinary Gains (Losses)',
 'Net Income',
 'Net Income (Common)',
 'Periodo',
 'Price_d',
 'Price_d_100',
 'Publish_Date+180',
 'Price_d+180',
 'Price_d+180_100',
 'Indice_d_100',
 'Indi

### Capitalización bursátil

In [7]:
data['Capit']=data['Shares (Basic)']*data['Price_d']
len(data[data['Capit'].isnull()])

30

### PER

In [8]:
data['PER'] = data['Capit']/data['Net Income (Common)']
len(data[data['PER'].isnull()])

30

### Price / Book Value

In [9]:
data['PBC'] = data['Capit']/data['Total Equity']
len(data[data['PBC'].isnull()])

30

### ROE

In [10]:
data['ROE'] = data['Net Income (Common)']/data['Total Equity']
len(data[data['ROE'].isnull()])

0

### ROA

In [11]:
data['ROA'] = data['Net Income (Common)']/data['Total Assets']
len(data[data['ROA'].isnull()])

0

### Deuda total / FFPP

In [12]:
data['Deuda_FFPP'] = (data['Short Term Debt'] + data['Long Term Debt'])/data['Total Equity']
len(data[data['Deuda_FFPP'].isnull()])

252

### Deuda corto plazo / FFPP

In [13]:
data['DeudaCP_FFPP'] = data['Short Term Debt'] / data['Total Equity']
len(data[data['DeudaCP_FFPP'].isnull()])

101

### Dotación en el periodo / Total Activo

In [14]:
data['Dot_dudosos'] = data['Provision for Loan Losses']/data['Total Assets']
len(data[data['Dot_dudosos'].isnull()])

39

## Variaciones en los ratios

A continuación vamos a incorporar como variable la variación de cada ratio con respecto al periodo anterior. Para ello primero aseguramos el orden adecuado de la tabla

In [15]:
data.sort_values(['Ticker','Fiscal Year','Fiscal Period'], ascending=True, inplace=True)

Despues incorporamos en cada fila el valor del ratio del periodo anterior, siempre que el Ticker no varíe.

In [16]:
#Definimos una función que genere subtablas por Ticker y en ellas haga un shift para incorporar en cada línea el
#valor de un ratio dado del periodo anterior.

def ratio_ant(ratio,data):
    table_1=pd.DataFrame()
    for ticker in data.Ticker.unique():
        aux_table=data[data.Ticker==ticker]
        aux_table['var_%s'%(ratio)]=(aux_table[ratio].shift(1)/aux_table[ratio])-1
        table_1=table_1.append(aux_table)
    return table_1

In [17]:
#Aplicamos la función anterior para cada ratio

for column in ['PER','PBC','ROE','ROA','Deuda_FFPP','DeudaCP_FFPP','Dot_dudosos']:
    data=ratio_ant(column,data)

/home/pipe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Valor relativo de cada ratio

Una posible mejora del modelo consiste en considerar cada uno de los ratios no por su valor absoluto, sino como un valor relativo con respecto al resto de compañias del periodo.
Para ello vamos a obtener para cada periodo el percentil que ocupa cada ratio de cada compañía con respecto a las demás.

In [18]:
data.head(4)

,Ticker,Fiscal Year,Fiscal Period,Publish Date,Shares (Basic),Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Interbank Assets,Short & Long Term Investments,Accounts & Notes Receivable,Net Loans,Net Fixed Assets,Total Assets,Total Deposits,Short Term Debt,Long Term Debt,Total Liabilities,Preferred Equity,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Revenue,Provision for Loan Losses,Net Revenue after Provisions,Total Non-Interest Expense,Operating Income (Loss),Non-Operating Income (Loss),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Periodo,Price_d,Price_d_100,Publish_Date+180,Price_d+180,Price_d+180_100,Indice_d_100,Indice_d+180_100,Var_cotiz,Var_index,Etiqueta,Capit,PER,PBC,ROE,ROA,Deuda_FFPP,DeudaCP_FFPP,Dot_dudosos,var_PER,var_PBC,var_ROE,var_ROA,var_Deuda_FFPP,var_DeudaCP_FFPP,var_Dot_dudosos
0,C,2011,Q2,2011-08-05,2.908600e+09,2.997000e+09,27766000000,4.401570e+11,631923000000,4.069500e+10,6.131380e+11,NaN,1956626000000,8.663100e+11,4.862840e+11,3.524580e+11,1777981000000,312000000.0,103504000000,-1.087000e+09,8.585700e+10,178645000000,1956626000000,20622000000,-3.387000e+09,17235000000,-12936000000,4299000000,NaN,4299000000,-9.670000e+08,3332000000,71000000.0,3341000000,3341000000,2011Q2,30.33,75.185920,2012-02-01,28.67,71.070897,86.447130,96.574366,-0.054731,0.117149,Peor,8.821784e+10,26.404621,0.493816,0.018702,0.001708,4.695021,2.722069,-0.001731,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C,2011,Q3,2011-11-04,2.910800e+09,2.998600e+09,28950000000,4.499830e+11,607294000000,3.799200e+10,6.051870e+11,NaN,1935992000000,8.512810e+11,4.943740e+11,3.338240e+11,1756650000000,312000000.0,105591000000,-1.089000e+09,8.960200e+10,179342000000,1935992000000,20831000000,-3.351000e+09,17480000000,-12460000000,5020000000,NaN,5020000000,-1.278000e+09,3742000000,1000000.0,3771000000,3771000000,2011Q3,27.53,68.244918,2012-05-02,29.68,73.574616,86.284868,102.828194,0.078097,0.191729,Peor,8.013432e+10,21.250152,0.446824,0.021027,0.001948,4.617981,2.756599,-0.001731,0.242561,0.105170,-0.110571,-0.123371,0.016683,-0.012526,0.000084
2,C,2011,Q4,2012-02-24,2.915400e+09,3.003000e+09,28701000000,4.316330e+11,585147000000,2.777700e+10,6.171270e+11,NaN,1873878000000,8.659360e+11,4.355920e+11,3.235050e+11,1694305000000,312000000.0,105833000000,-1.071000e+09,9.052000e+10,179573000000,1873878000000,16152000000,-2.437000e+09,13715000000,-12528000000,1187000000,NaN,1187000000,-1.450000e+08,1042000000,-44000000.0,956000000,956000000,2011Q4,29.36,72.781358,2012-08-22,27.69,68.641547,97.810615,101.998539,-0.056880,0.042817,Peor,8.559614e+10,89.535715,0.476665,0.005324,0.000510,4.227233,2.425710,-0.001301,-0.762663,-0.062603,2.949641,2.818004,0.092436,0.136409,0.330934
3,C,2012,Q1,2012-05-04,2.926200e+09,3.014500e+09,26505000000,4.730060e+11,604373000000,3.944300e+10,6.190020e+11,NaN,1944423000000,9.060120e+11,4.745410e+11,3.110790e+11,1760700000000,312000000.0,105816000000,-8.830000e+08,9.331000e+10,183723000000,1944423000000,19406000000,-3.019000e+09,16387000000,-12319000000,4068000000,NaN,4068000000,-1.006000e+09,3062000000,-5000000.0,2931000000,2931000000,2012Q1,28.69,71.120476,2012-10-31,33.96,84.184432,101.357729,104.113830,0.183688,0.027192,Mejor,8.395268e+10,28.643015,0.456952,0.015953,0.001507,4.276111,2.582916,-0.001553,2.125918,0.043139,-0.666294,-0.661552,-0.011430,-0.060864,-0.162390


In [19]:
data = data.astype({'Fiscal Year':str})
data['Periodo'] = data['Fiscal Year'] + data['Fiscal Period']

In [20]:
len(data)

1021

In [21]:
#Definimos una función que genere subtablas por periodo y cada una calcule el cuantil de cada ratio de cada
#compañia

In [22]:
from scipy import stats

def my_quantile(ratio,data):
    table_1=pd.DataFrame()
    
    for periodo in data.Periodo.unique():
        aux_table=data[data.Periodo==periodo]
        aux_table['quantile_%s'%(ratio)]=0       
        
        for row in range(0,len(aux_table)):
            aux_table['quantile_%s'%(ratio)].iloc[row]=stats.percentileofscore(aux_table[ratio],
                                                                               aux_table[ratio].iloc[row])
        
        table_1=table_1.append(aux_table)
            
    return table_1

In [23]:
#Aplicamos la función anterior para cada ratio

for column in ['PER','PBC','ROE','ROA','Deuda_FFPP','DeudaCP_FFPP','Dot_dudosos']:
    data=my_quantile(column,data)

/home/pipe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pipe/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1942: RuntimeWarning: invalid value encountered in less
  left = np.count_nonzero(a < score)
/home/pipe/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1943: RuntimeWarning: invalid value encountered in less_equal
  right = np.count_nonzero(a <= score)
/home/pipe/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

## Variaciones en los cuantiles

A continuación vamos a incorporar como variable la variación de cada percentil con respecto al periodo anterior. Para ello primero aseguramos el orden adecuado de la tabla

In [24]:
data.sort_values(['Ticker','Fiscal Year','Fiscal Period'], ascending=True, inplace=True)

Despues incorporamos en cada fila el valor del ratio del periodo anterior, siempre que el Ticker no varíe.

In [25]:
#Definimos una función que genere subtablas por Ticker y en ellas haga un shift para incorporar en cada línea el
#valor de un cuantil dado del periodo anterior.

def ratio_ant(ratio,data):
    table_1=pd.DataFrame()
    for ticker in data.Ticker.unique():
        aux_table=data[data.Ticker==ticker]
        aux_table['var_%s'%(ratio)]=(aux_table[ratio]-aux_table[ratio].shift(1))
        table_1=table_1.append(aux_table)
    return table_1

In [26]:
#Aplicamos la función anterior para cada ratio

for column in ['quantile_PER','quantile_PBC','quantile_ROE','quantile_ROA','quantile_DeudaCP_FFPP','quantile_Dot_dudosos']:
    data=ratio_ant(column,data)

/home/pipe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
data[data.Ticker=='C'].head(10)

,Ticker,Fiscal Year,Fiscal Period,Publish Date,Shares (Basic),Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Interbank Assets,Short & Long Term Investments,Accounts & Notes Receivable,Net Loans,Net Fixed Assets,Total Assets,Total Deposits,Short Term Debt,Long Term Debt,Total Liabilities,Preferred Equity,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Revenue,Provision for Loan Losses,Net Revenue after Provisions,Total Non-Interest Expense,Operating Income (Loss),Non-Operating Income (Loss),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Periodo,Price_d,Price_d_100,Publish_Date+180,Price_d+180,Price_d+180_100,Indice_d_100,Indice_d+180_100,Var_cotiz,Var_index,Etiqueta,Capit,PER,PBC,ROE,ROA,Deuda_FFPP,DeudaCP_FFPP,Dot_dudosos,var_PER,var_PBC,var_ROE,var_ROA,var_Deuda_FFPP,var_DeudaCP_FFPP,var_Dot_dudosos,quantile_PER,quantile_PBC,quantile_ROE,quantile_ROA,quantile_Deuda_FFPP,quantile_DeudaCP_FFPP,quantile_Dot_dudosos,var_quantile_PER,var_quantile_PBC,var_quantile_ROE,var_quantile_ROA,var_quantile_DeudaCP_FFPP,var_quantile_Dot_dudosos
0,C,2011,Q2,2011-08-05,2.908600e+09,2.997000e+09,27766000000,4.401570e+11,631923000000,4.069500e+10,6.131380e+11,NaN,1956626000000,8.663100e+11,4.862840e+11,3.524580e+11,1777981000000,3.120000e+08,103504000000,-1.087000e+09,8.585700e+10,178645000000,1956626000000,20622000000,-3.387000e+09,17235000000,-12936000000,4299000000,NaN,4299000000,-9.670000e+08,3332000000,71000000.0,3341000000,3341000000,2011Q2,30.33,75.185920,2012-02-01,28.67,71.070897,86.447130,96.574366,-0.054731,0.117149,Peor,8.821784e+10,26.404621,0.493816,0.018702,0.001708,4.695021,2.722069,-0.001731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.470588,14.705882,44.117647,35.294118,70.588235,85.294118,20.588235,NaN,NaN,NaN,NaN,NaN,NaN
1,C,2011,Q3,2011-11-04,2.910800e+09,2.998600e+09,28950000000,4.499830e+11,607294000000,3.799200e+10,6.051870e+11,NaN,1935992000000,8.512810e+11,4.943740e+11,3.338240e+11,1756650000000,3.120000e+08,105591000000,-1.089000e+09,8.960200e+10,179342000000,1935992000000,20831000000,-3.351000e+09,17480000000,-12460000000,5020000000,NaN,5020000000,-1.278000e+09,3742000000,1000000.0,3771000000,3771000000,2011Q3,27.53,68.244918,2012-05-02,29.68,73.574616,86.284868,102.828194,0.078097,0.191729,Peor,8.013432e+10,21.250152,0.446824,0.021027,0.001948,4.617981,2.756599,-0.001731,0.242561,0.105170,-0.110571,-0.123371,0.016683,-0.012526,0.000084,14.705882,8.823529,52.941176,38.235294,70.588235,85.294118,14.705882,-11.764706,-5.882353,8.823529,2.941176,0.000000,-5.882353
2,C,2011,Q4,2012-02-24,2.915400e+09,3.003000e+09,28701000000,4.316330e+11,585147000000,2.777700e+10,6.171270e+11,NaN,1873878000000,8.659360e+11,4.355920e+11,3.235050e+11,1694305000000,3.120000e+08,105833000000,-1.071000e+09,9.052000e+10,179573000000,1873878000000,16152000000,-2.437000e+09,13715000000,-12528000000,1187000000,NaN,1187000000,-1.450000e+08,1042000000,-44000000.0,956000000,956000000,2011Q4,29.36,72.781358,2012-08-22,27.69,68.641547,97.810615,101.998539,-0.056880,0.042817,Peor,8.559614e+10,89.535715,0.476665,0.005324,0.000510,4.227233,2.425710,-0.001301,-0.762663,-0.062603,2.949641,2.818004,0.092436,0.136409,0.330934,82.352941,5.882353,14.705882,8.823529,73.529412,88.235294,20.588235,67.647059,-2.941176,-38.235294,-29.411765,2.941176,5.882353
3,C,2012,Q1,2012-05-04,2.926200e+09,3.014500e+09,26505000000,4.730060e+11,604373000000,3.944300e+10,6.190020e+11,NaN,1944423000000,9.060120e+11,4.745410e+11,3.110790e+11,1760700000000,3.120000e+08,105816000000,-8.830000e+08,9.331000e+10,183723000000,1944423000000,19406000000,-3.019000e+09,16387000000,-12319000000,4068000000,NaN,4068000000,-1.006000e+09,3062000000,-5000000.0,2931000000,2931000000,2012Q1,28.69,71.120476,2012-10-31,33.96,84.184432,101.357729,104.113830,0.183688,0.027192,Mejor,8.395268e+10,28.643015,0.456952,0.01

In [28]:
data.to_csv('../tablas/dataformodel.csv', index=False)